In [1]:
import pandas as pd
import matplotlib.pyplot as plt # Visualization
import sunpy
import math
import numpy as np
from sunpy.coordinates.sun import carrington_rotation_number
from scipy.spatial import distance

In [2]:
import warnings
warnings.filterwarnings('ignore')

Greenwich data from: https://solarscience.msfc.nasa.gov/greenwch.shtml \
Zurich data from: https://github.com/observethesun/zurich_catalogs

In [3]:
import glob
dfs = []
for fin in glob.glob('greenwich/*.txt'):
    dfs.append(pd.read_csv(fin, header = None))
greenwich = pd.concat(dfs)

dfs = []
for fin in glob.glob('Digitized_*.csv'):
    dfs.append(pd.read_csv(fin))
dig = pd.concat(dfs, ignore_index=True)

In [4]:
def is_float(element: any) -> bool:
    #If you expect None to be passed:
    if element is None: 
        return False
    try:
        float(element)
        return True
    except ValueError:
        return False


In [5]:
dig = dig.loc[dig["Object"] == 'Sunspot']
dig['Date'] = pd.to_datetime(dig['Date'])
dig["CR"] = carrington_rotation_number(dig.loc[:, "Date"])
dig = dig.dropna()
    
greenwich["year"] = greenwich[0].str.slice(stop = 4).astype(int)
greenwich["month"] = greenwich[0].str.slice(start = 4, stop = 6).astype(int)
greenwich["day"] = greenwich[0].str.slice(start = 6, stop = 8).astype(int)
greenwich["Greenwich sunspot group"] = greenwich[0].str.slice(start = 12, stop = 20).apply(lambda x: int(x) 
    if (x.replace(" ","").isnumeric()) else None)
greenwich["CLongtitude"] = greenwich[0].str.slice(start = 57, stop = 62).apply(lambda x: float(x) 
    if is_float(x.replace(" ","")) else None).astype(float)
greenwich["Latitude"] = greenwich[0].str.slice(start = 63, stop = 68).apply(lambda x: float(x) 
    if is_float(x.replace(" ","")) else None).astype(float)
greenwich["Date"] = pd.to_datetime(greenwich[0].str.slice(stop = 4) + "-" + greenwich["month"].astype(str) + "-" + greenwich["day"].astype(str))
greenwich["CR"] = carrington_rotation_number(greenwich.loc[:, "Date"])
greenwich = greenwich.dropna().reset_index(drop=True)


In [6]:
def new_metric(x, y):
    dist = math.sqrt(min(abs(x[0] - y[0]), abs(360 - max(x[0],y[0]) + min(x[0],y[0]))) ** 2 + (y[1] - x[1]) ** 2)
    return dist

def close_points(x, a = -0.78932204, b = 6.62623414):
    return  1/(1 + np.exp(-(a*x +b)))

def find_id(X, y):
    dist = distance.cdist(X, y, metric=new_metric)
    
    return np.argmin(dist)


def algorithm(X, C, cl_numbers):
    """
    X - точки
    C - центры групп
    
    points - номера кластеров для каждой точки
    rankings - величина R для каждой точки Х
    """
    Y = C.copy()
    X2 = X.copy()
    points =  np.full(X.shape[0], -1)
    rankings = np.zeros(C.shape[0] + X.shape[0])
    rankings[:C.shape[0]] = 1
    
    # Восстановление позиции в X
    positions = np.arange(X.shape[0])
    por = np.full(X.shape[0], -1)
    
    for it in range(X.shape[0]):
        dist = distance.cdist(X, Y, metric=new_metric)
        pr = close_points(dist)

        pr *= np.append(rankings[:C.shape[0]], rankings[por[:it] + C.shape[0]])
        k = np.argmax(pr)
        i, j = k // pr.shape[1], k % pr.shape[1]
        
        if (j < C.shape[0]):
            points[positions[i]] = cl_numbers[j]
        else:
            points[positions[i]] = points[por[j - C.shape[0]]]   
            
        Y = np.append(Y, X[i][None, :], axis=0)
        por[it] = find_id(X2, X[i].reshape(1, 2))
        rankings[positions[i] + C.shape[0]] = np.max(pr)
        X = np.delete(X, i, axis=0)
        positions = np.delete(positions, i)
    return points, rankings

In [7]:
from tqdm import tqdm
all_days = []
for idx, day in tqdm(dig.groupby(dig.Date)):
    date = day["Date"].iloc[0]
    day_gr = greenwich.loc[greenwich["Date"] == date]
    if (day_gr.shape[0] == 0):
        continue
    s = day_gr["Greenwich sunspot group"].unique()
    greenwich.loc[greenwich["Date"] == date + pd.Timedelta("1 day")]

    s = list(day_gr["Greenwich sunspot group"])
    indexes = []

    for key, value in greenwich.loc[greenwich["Date"] == date + pd.Timedelta("1 day")]["Greenwich sunspot group"].items():
        if value not in s:
            indexes.append(key)

    for key, value in greenwich.loc[greenwich["Date"] == date - pd.Timedelta("1 day")]["Greenwich sunspot group"].items():
        if value not in s:
            indexes.append(key)
    day_gr = pd.concat([day_gr, greenwich.iloc[indexes]])

    day = dig.loc[dig["Date"] == date].dropna()
    if (day.shape[0] == 0):
        raise Exception('No data')

    day.loc[:, "Column_6"] -= (68.70553089 - 0.08209453 * (date -  pd.to_datetime("1880-01-01")).days) % 360
    day.loc[:, "Column_6"]  = day.loc[:, "Column_6"].apply(lambda x: x + 360 if (x < 0) else x)
    points, rankings = algorithm(day.loc[:, ["Column_6", "Column_4"]].to_numpy(),
                                 day_gr.loc[:, ["CLongtitude", "Latitude"]].to_numpy(),
                                day_gr.loc[:, ["Greenwich sunspot group"]].to_numpy())

    rankings = np.round(rankings[day_gr.shape[0]:], 2)
    day.loc[:, "clusters"] = -1
    day.loc[:, "rankings"] = -1

    for i in range(day.shape[0]):
        day.iloc[i, [22]] = points[i]
        day.iloc[i, [23]] = rankings[i]
    all_days.append(day)



100%|███████████████████████████████████████| 6725/6725 [04:19<00:00, 25.88it/s]


In [8]:
data = pd.concat(all_days)

In [9]:
data

,Date,Object,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Confidence_1,Confidence_2,...,Regression_2,Regression_3,Regression_5,Regression_6,Source,Row,Empty,CR,clusters,rankings
318634,1887-01-26,Sunspot,-1.03,42.21,99.11,-2.01,97.93,50.422546,1.00,1.0,...,False,False,False,False,Z1887-1892_Page_0002,1,False,445.723891,1950,1.00
318635,1887-01-26,Sunspot,-3.35,51.54,89.78,-3.30,89.36,41.852546,1.00,1.0,...,False,False,False,False,Z1887-1892_Page_0002,2,False,445.723891,1949,1.00
318636,1887-01-26,Sunspot,-3.42,52.21,89.11,-3.28,88.70,41.192546,0.67,1.0,...,False,False,False,False,Z1887-1892_Page_0002,3,False,445.723891,1949,1.00
318637,1887-01-26,Sunspot,-3.20,56.82,84.50,-2.50,84.15,36.642546,1.00,1.0,...,False,False,False,False,Z1887-1892_Page_0002,4,False,445.723891,1949,0.95
318638,1887-01-26,Sunspot,-4.88,61.98,79.34,-3.55,78.83,31.322546,1.00,1.0,...,False,False,False,False,Z1887-1892_Page_0002,5,False,445.723891,1951,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659618,1920-12-31,Sunspot,-8.40,-8.30,122.40,-12.00,121.40,341.477961,1.00,1.0,...,False,False,False,False,Z1916-1920_Page_2249,59,False,900.057910,9280,0.94
659619,1920-12-31,Sunspot,-7.10,-7.90,122.00,-10.70,121.10,341.177961,1.00,1.0,...,False,False,False,False,Z1916-1920_Page_2249,60,False,900.057910,9280,0.93
659620,1920-12-31,Sunspot,-12.10,71.10,43.00,-7.00,42.20,262.277961,1.00,1.0,...,False,False,False,False,Z1916-1920_Page_2249,61,False,900.057910,9282,0.93
659621,1920-12-31,Sunspot,-15.20,80.00,34.10,-9.40,33.20,253.277961,1.00,1.0,...,False,False,False,False,Z1916-1920_Page_2249,62,False,900.057910,9282,0.97


In [ ]:
data.to_csv("new_data.csv")